# **SET FRAME WIDTH as per EPHYS TIMESTAMP**

**Notes:** try to set frame width with 
set(cv2.CAP_PROP_FRAME_WIDTH)... smt like that

https://docs.opencv.org/3.4/d8/dfe/classcv_1_1VideoCapture.html#a473055e77dd7faa4d26d686226b292c1

In [5]:
import os
#os.chdir("../../")
os.listdir()

['.ipynb_checkpoints',
 'calculate_video_timepoint.py',
 'motion_detector_ephysTime.ipynb',
 'plot_mvm.py',
 'vid_processing.ipynb',
 'video_process_pck',
 'conv_dat2eeg.sh',
 'readDat_SYNC.ipynb',
 'file_cleaner.ipynb',
 'conv_dat2eeg.ipynb',
 'vid_processing-Copy1.ipynb',
 'CSV_data',
 'imro.ipynb']

In [6]:
from video_process_pck.motion_detector import *

In [74]:
#try: cap_prop_pos_msec... https://www.quora.com/How-do-I-decrease-the-frames-per-second-in-OpenCV-python

def motion_detector(path, scale_percent=40, area=20, delta_thresh=5,
                    output_4csv="/home/domi/Documents/video_processing/CSV_data",
                   ephys_time_path="/media/data-119/rat596_20210701_184333/rat596_20210701_184333tmstp.txt"):
    """
    path: path to the video
    area: minimum area size.... TO INCLUDE!!!
    ephys_time: path to ephys timestamp.. it will load it and calculate the differences (np.diff()) 
        -> iterable input for frame width (CAP_PROP_FRAME_WIDTH)
    """
    # 1) EPHYS
    #read file
    e = open(ephys_time_path,"r")
    ephys_time = [[int(x) for x in line.split()] for line in e]
    e.close()
    ephys_time = np.concatenate(ephys_time)
    
    #convert eph to MILLIseconds (-> then diff it -> NO NEED TO DO THIS IF  setting POS_MSEC)
    ephys_time = ephys_time/2.5           # CORRECTION (#divide by sampling freq -> in secs now)
                                         # don't leave it in sec.. should be in millisec (/2500 *1000)? 
    diff_ephys_time = np.diff(ephys_time)   #exact TIME differences between frames... seems like i can't use this
    ephys_time_0 = [i - ephys_time[0] for i in ephys_time]
    
    
    # 2) OUTPUT
    #if specified output folder doesn't exist, create it
#    if not os.path.exists(output_4csv):
#        os.mkdir(output_4csv)

    # 3) VID_PROC
    # read from a video file 
    vs = cv2.VideoCapture(path)
    
    #initiate FramePerSec (FPS) count
    fps = FPS().start()

    # initialize average frame, frame delta and thresholded frame
    avgframe = [[], []]
    frameDelta = [[], []]
    thresh = [[], []]

    # initiate dictionary
    ts_dict = {"frame_nb": [], "millisec": [], "mvm_rat1": [], "mvm_rat2": []}

    # initiate dimensions for splitting
    # n_rows=1
    n_cols = 2  # 2 rats = 2 parts split with a vertical line
    # division in the middle

    # loop over the frames of the video
    n = 0
    
    #DELETE AFTER___________________________________________________________________________________________
    milliseconds = []
    print(ephys_time[0])
    #________________________________________________________________________________________________________
    
    while True:
        
        #SET FRAME WIDTH
        try:
            vs.set(cv2.CAP_PROP_POS_MSEC, ephys_time_0[n])
        except IndexError:
            print("More frames than ephys")
            pass
        
        ret, frame = vs.read()  # ret says whether the frame exists

        if frame is None:
            break  # end of video if no more frames

        # get timestamp of each frame...
        frame_nb = n
        millisec = str(vs.get(cv2.CAP_PROP_POS_MSEC))  # in millisecond
        
        #DELETE AFTER___________________________________________________________________________________________
        milliseconds.append(vs.get(cv2.CAP_PROP_POS_MSEC))
        #_________________________________________________________________________________________________________
        
        ts_dict = timestamping(ts_dict=ts_dict,
                               frame_nb=frame_nb,
                               millisec=millisec)

        n += 1
        # resize the frame, convert it to grayscale, and blur it
        frame = resizing(frame=frame, scale_percent=scale_percent)
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        gray = cv2.GaussianBlur(gray, (21, 21), 0)

        # get gray's shape for splitting............................................
        height, width = gray.shape

        # divide the frame and calculate background and change for each part
        for i in range(n_cols):
            tmp_img = gray[0:height, int(i * width / 2): int((i + 1) * width / 2)]  # take all hight and half of width

            # initialize average frame (=background)
            if len(avgframe[i]) == 0:
                avgframe[i] = tmp_img.copy().astype("float")

            # update average & calculate difference between current and running avg
            cv2.accumulateWeighted(tmp_img, avgframe[i], 0.5)
            frameDelta[i] = cv2.absdiff(tmp_img,
                                        cv2.convertScaleAbs(avgframe[i]))

            # threshold the delta image, dilate the thresholded image to fill
            # in holes, then find contours on thresholded image
            thresh[i] = cv2.threshold(frameDelta[i], delta_thresh, 255,
                                      cv2.THRESH_BINARY)[1]
            thresh[i] = cv2.dilate(thresh[i], None, iterations=1)  # spread the white pixels

        # append mvm count for each rat
        for i in range(2):
            ts_dict[f"mvm_rat{i + 1}"].append(np.sum(thresh[i]))

        # update FPS counter
        fps.update()

        # DISPLAY VIDEOS (IT'S 2x LONGER THIS WAY... 92:177 FPS)!!!
        # cv2.imshow("webcam", frame)
        # cv2.imshow("Thresh_rat1", thresh[0])
        # cv2.imshow("Thresh_rat2", thresh[1])
        # cv2.imshow("Frame Delta_rat2", frameDelta[1])

        # cv2.waitKey(1)
        keyboard = cv2.waitKey(1)
        if keyboard == 'q' or keyboard == 27:
            break

        # testing, remove after...................................................................................
#        if n > 25 * 60 * 1:
            #print(milliseconds[n-1])
        
    cv2.destroyAllWindows()
#           newpath, first_stamp = csv_name_creator(path, output_folder=output_4csv)
    ts_df = pd.DataFrame(ts_dict)

    ts_df['duration'] = [timedelta(milliseconds=float(i)) for i in ts_df['millisec']]
    ts_df['duration'] = ts_df['duration'].values.astype('datetime64[ns]')
    ts_df['duration'] = [i.strftime("%H:%M:%S.%f")[:-3] for i in ts_df['duration']]

#           ts_df['timestamp'] = [pd.to_timedelta(str(i) + 'millisecond') +
#                                 first_stamp for i in ts_df['millisec']]
#           ts_df.set_index('timestamp', inplace=True)

#          order = [0, 1, 4, 2, 3]  # setting column's order
#          ts_df = ts_df[[ts_df.columns[i] for i in order]]

# ts_df.to_csv(csv_name_creator(path, output_folder=output_4csv)[0], sep='\t')

# stop the timer and display FPS information
    fps.stop()
    print("[INFO] elasped time: {:.2f}".format(fps.elapsed()))
    print("[INFO] approx. FPS: {:.2f}".format(fps.fps()))

# return the df with timestamp+mvm count
#            return ts_df#..........................................................................................
    return milliseconds



    # stop the timer and print Frame Per Sec info
    fps.stop()
    print("[INFO] elapsed time: {:.2f}".format(fps.elapsed()))
    print("[INFO] approx. FPS: {:.2f}".format(fps.fps()))

    # CLEAN UP
    cv2.destroyAllWindows()

    # get path for csv output & init of timestamp (extracted from the video filename)
    newpath, first_stamp = csv_name_creator(path, output_folder=output_4csv)

    # convert dict do pandas data frame...
    ts_df = pd.DataFrame(ts_dict)

    # create column 'duration' (millisec converted to time)
    ts_df['duration'] = [timedelta(milliseconds=float(i)) for i in ts_df['millisec']]
    # to keep only the time of the timedelta (otherwise give you estimate with days as well)
    ts_df['duration'] = ts_df['duration'].values.astype('datetime64[ns]')
    ts_df['duration'] = [i.strftime("%H:%M:%S.%f")[:-3] for i in ts_df['duration']]

    # ...set exact datetime timestamp (adding the first stamp to milliseconds)...
    ts_df['timestamp'] = [pd.to_timedelta(str(i) + 'millisecond') +
                          first_stamp for i in ts_df['millisec']]
    ts_df.set_index('timestamp', inplace=True)

    order = [0, 1, 4, 2, 3]  # setting columns' order
    ts_df = ts_df[[ts_df.columns[i] for i in order]]

    # ...and save it to info directory
    ts_df.to_csv(newpath, sep='\t')
    # return ts_df

In [75]:
df_nn=motion_detector("/media/data-119/rat596_20210701_184333/rat596_20210701_184333.mp4")


4022.4


KeyboardInterrupt: 

In [48]:
df_nn_sec=[i/1000 for i in df_nn]

In [50]:
len(df_nn)

1336934

In [52]:
nnn=np.diff(df_nn_sec)


In [53]:
nnn


array([0.04, 0.04, 0.04, ..., 0.  , 0.  , 0.  ])